<a href="https://colab.research.google.com/github/SomeDieYoung27/Sarvam/blob/main/csls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install gensim==4.3.3
!pip install matplotlib
!pip install scikit-learn
import numpy as np

In [3]:

!wget -c https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
!wget -c https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.hi.300.vec.gz


--2024-09-24 15:54:36--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.78, 13.226.210.25, 13.226.210.111, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.78|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

--2024-09-24 15:54:37--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.hi.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.78, 13.226.210.25, 13.226.210.111, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.78|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [14]:
MAX_VOCAB_SIZE = 10000  # Adjust based on available memory
EMBEDDING_DIM = 300
BATCH_SIZE = 64  # Adjust based on memory constraints
N_EPOCHS = 5  # Number of training epochs
K_CSLS = 10  # Number of nearest neighbors for CSLS
K_PRECISION = [1, 5]  # Precision@1 and Precision@5

In [15]:
!wget -c https://dl.fbaipublicfiles.com/arrival/dictionaries/en-hi.0-5000.txt
!wget -c https://dl.fbaipublicfiles.com/arrival/dictionaries/en-hi.5000-6500.txt

--2024-09-24 17:28:26--  https://dl.fbaipublicfiles.com/arrival/dictionaries/en-hi.0-5000.txt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.78, 13.226.210.25, 13.226.210.15, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.78|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

--2024-09-24 17:28:27--  https://dl.fbaipublicfiles.com/arrival/dictionaries/en-hi.5000-6500.txt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.78, 13.226.210.25, 13.226.210.15, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.78|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [16]:
def load_bilingual_lexicon(file_path):
    bilingual_dict = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 2:
                bilingual_dict.append((parts[0].lower(), parts[1].lower()))
    return bilingual_dict

In [7]:
train_dict = load_bilingual_lexicon('en-hi.0-5000.txt')
test_dict = load_bilingual_lexicon('en-hi.5000-6500.txt')

In [8]:
!gunzip -k cc.en.300.vec.gz
!gunzip -k cc.hi.300.vec.gz

In [17]:
def load_relevant_embeddings(embedding_file, relevant_words, max_vocab_size):
    embeddings = {}
    with open(embedding_file, 'r', encoding='utf-8', errors='ignore') as f:
        next(f)  # Skip header
        for line in f:
            parts = line.rstrip().split(' ')
            word = parts[0].lower()
            if word in relevant_words:
                vector = np.array(parts[1:], dtype=np.float32)
                embeddings[word] = vector
                if len(embeddings) >= max_vocab_size:
                    break
    return embeddings

In [18]:
def collect_unique_words(bilingual_dicts):
    src_words = set()
    tgt_words = set()
    for bilingual_dict in bilingual_dicts:
        for src_word, tgt_word in bilingual_dict:
            src_words.add(src_word)
            tgt_words.add(tgt_word)
    return src_words, tgt_words

In [13]:
src_words, tgt_words = collect_unique_words([train_dict, test_dict])

# Load embeddings
en_embeddings_dict = load_relevant_embeddings('cc.en.300.vec', src_words, MAX_VOCAB_SIZE)
hi_embeddings_dict = load_relevant_embeddings('cc.hi.300.vec', tgt_words, MAX_VOCAB_SIZE)

In [21]:
# Create KeyedVectors
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from gensim.models import KeyedVectors
import gc
from gensim.models import KeyedVectors

def create_keyed_vectors(embeddings_dict):
    kv = KeyedVectors(vector_size=EMBEDDING_DIM)
    kv.add_vectors(list(embeddings_dict.keys()), list(embeddings_dict.values()))
    return kv

en_embeddings = create_keyed_vectors(en_embeddings_dict)
hi_embeddings = create_keyed_vectors(hi_embeddings_dict)

# Step 5: Prepare Embeddings for Training
# Convert embeddings to torch tensors
src_embeddings = torch.from_numpy(en_embeddings.vectors).float()
tgt_embeddings = torch.from_numpy(hi_embeddings.vectors).float()

# Normalize embeddings
src_embeddings = src_embeddings / src_embeddings.norm(2, dim=1, keepdim=True)
tgt_embeddings = tgt_embeddings / tgt_embeddings.norm(2, dim=1, keepdim=True)

In [22]:
class Discriminator(nn.Module):
    def __init__(self, input_dim, hidden_dim=2048):
        super(Discriminator, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, 1)
        )
    def forward(self, x):
        return self.layer(x)

In [23]:
mapping = nn.Linear(EMBEDDING_DIM, EMBEDDING_DIM, bias=False)
nn.init.eye_(mapping.weight)
discriminator = Discriminator(EMBEDDING_DIM)

# Move models to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mapping = mapping.to(device)
discriminator = discriminator.to(device)
src_embeddings = src_embeddings.to(device)
tgt_embeddings = tgt_embeddings.to(device)

# Step 7: Set Up Optimizers
lr = 0.1
mapping_optimizer = optim.SGD(mapping.parameters(), lr=lr)
discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))


In [24]:
for epoch in range(N_EPOCHS):
    # Shuffle indices
    indices = torch.randperm(src_embeddings.size(0))
    num_batches = src_embeddings.size(0) // BATCH_SIZE

    for i in range(num_batches):
        # Get batch indices
        batch_indices = indices[i * BATCH_SIZE:(i + 1) * BATCH_SIZE]
        # Get source and target batch embeddings
        src_batch = src_embeddings[batch_indices]
        tgt_batch = tgt_embeddings[batch_indices]

        # Train discriminator
        discriminator_optimizer.zero_grad()
        # Map source embeddings
        src_mapped = mapping(src_batch)
        # Discriminator outputs
        src_preds = discriminator(src_mapped.detach())
        tgt_preds = discriminator(tgt_batch)
        # Labels
        src_labels = torch.zeros(src_batch.size(0), 1).to(device)
        tgt_labels = torch.ones(tgt_batch.size(0), 1).to(device)
        # Loss
        d_loss_src = nn.functional.binary_cross_entropy_with_logits(src_preds, src_labels)
        d_loss_tgt = nn.functional.binary_cross_entropy_with_logits(tgt_preds, tgt_labels)
        d_loss = d_loss_src + d_loss_tgt
        # Backpropagation
        d_loss.backward()
        discriminator_optimizer.step()

        # Train mapping (generator)
        mapping_optimizer.zero_grad()
        # Map source embeddings
        src_mapped = mapping(src_batch)
        # Discriminator output
        src_preds = discriminator(src_mapped)
        # Labels (want discriminator to think mapped embeddings are target)
        src_labels = torch.ones(src_batch.size(0), 1).to(device)
        # Loss
        g_loss = nn.functional.binary_cross_entropy_with_logits(src_preds, src_labels)
        # Backpropagation
        g_loss.backward()
        mapping_optimizer.step()

    print(f'Epoch {epoch + 1}/{N_EPOCHS}, Discriminator Loss: {d_loss.item():.4f}, Generator Loss: {g_loss.item():.4f}')


Epoch 1/5, Discriminator Loss: 0.0000, Generator Loss: 11410032.0000
Epoch 2/5, Discriminator Loss: 23.6154, Generator Loss: 53442344.0000
Epoch 3/5, Discriminator Loss: 0.0000, Generator Loss: 1268622950400.0000
Epoch 4/5, Discriminator Loss: 5409.4761, Generator Loss: 2264265654272.0000
Epoch 5/5, Discriminator Loss: 0.0000, Generator Loss: 907598364672.0000


In [25]:
W_adv = mapping.weight.data.cpu().numpy()

# Map source embeddings
en_mapped_embeddings = en_embeddings.vectors @ W_adv.T

# Normalize mapped embeddings
en_mapped_embeddings = en_mapped_embeddings / np.linalg.norm(en_mapped_embeddings, axis=1, keepdims=True)

In [26]:
def build_dictionary(src_emb, tgt_emb, src_words, tgt_words, k=1):
    similarities = src_emb @ tgt_emb.T
    nn_indices = np.argpartition(-similarities, range(k), axis=1)[:, :k]
    word_pairs = []
    for i, indices in enumerate(nn_indices):
        src_word = src_words[i]
        for idx in indices:
            tgt_word = tgt_words[idx]
            word_pairs.append((src_word, tgt_word))
    return word_pairs

# Build pseudo-dictionary
pseudo_dict = build_dictionary(en_mapped_embeddings, hi_embeddings.vectors, en_embeddings.index_to_key, hi_embeddings.index_to_key, k=1)
print(f'Pseudo-dictionary built with {len(pseudo_dict)} word pairs.')

Pseudo-dictionary built with 6484 word pairs.


In [27]:
def create_embedding_matrices(bilingual_dict, source_embeddings, target_embeddings):
    source_matrix = []
    target_matrix = []
    oov_count = 0
    for src_word, tgt_word in bilingual_dict:
        if src_word in source_embeddings.key_to_index and tgt_word in target_embeddings.key_to_index:
            source_matrix.append(source_embeddings[src_word])
            target_matrix.append(target_embeddings[tgt_word])
        else:
            oov_count += 1
    print(f'OOV pairs: {oov_count}')
    return np.array(source_matrix), np.array(target_matrix)

# Create matrices using pseudo-dictionary
X_train, Y_train = create_embedding_matrices(pseudo_dict, en_embeddings, hi_embeddings)

# Compute refined mapping
def compute_procrustes(X, Y):
    # Center the embeddings
    X_mean = X.mean(0)
    Y_mean = Y.mean(0)
    X -= X_mean
    Y -= Y_mean
    # Compute covariance matrix
    M = Y.T @ X
    # Singular Value Decomposition
    U, _, Vt = np.linalg.svd(M)
    # Compute orthogonal matrix W
    W = U @ Vt
    return W

W_refined = compute_procrustes(X_train, Y_train)

# Map English embeddings using refined mapping
en_mapped_embeddings = (en_embeddings.vectors - en_embeddings.vectors.mean(0)) @ W_refined.T + hi_embeddings.vectors.mean(0)

# Normalize mapped embeddings
en_mapped_embeddings = en_mapped_embeddings / np.linalg.norm(en_mapped_embeddings, axis=1, keepdims=True)

# Create KeyedVectors instance
en_mapped = KeyedVectors(vector_size=EMBEDDING_DIM)
en_mapped.add_vectors(en_embeddings.index_to_key, en_mapped_embeddings)


OOV pairs: 0


In [28]:
def compute_csls(src_emb, tgt_emb, k=10, batch_size=1024):
    # Initialize arrays to store average similarities
    src_avg_sim = np.zeros(src_emb.shape[0])
    tgt_avg_sim = np.zeros(tgt_emb.shape[0])

    # Compute source to target similarities in batches
    for i in range(0, src_emb.shape[0], batch_size):
        src_batch = src_emb[i:i+batch_size]
        sims = src_batch @ tgt_emb.T
        sorted_sims = np.sort(sims, axis=1)[:, -k:]
        src_avg_sim[i:i+batch_size] = np.mean(sorted_sims, axis=1)

    # Compute target to source similarities in batches
    for i in range(0, tgt_emb.shape[0], batch_size):
        tgt_batch = tgt_emb[i:i+batch_size]
        sims = tgt_batch @ src_emb.T
        sorted_sims = np.sort(sims, axis=1)[:, -k:]
        tgt_avg_sim[i:i+batch_size] = np.mean(sorted_sims, axis=1)

    return src_avg_sim, tgt_avg_sim

# Compute CSLS average similarities
src_avg_sim, tgt_avg_sim = compute_csls(en_mapped_embeddings, hi_embeddings.vectors, k=K_CSLS, batch_size=1024)

In [29]:
def precision_at_k_csls(source_embeddings, target_embeddings, test_dict, src_avg_sim, tgt_avg_sim, k=1, batch_size=1024):
    correct = 0
    total = 0
    src_vectors = source_embeddings.vectors
    tgt_vectors = target_embeddings.vectors

    # Build index mappings
    src_word2idx = source_embeddings.key_to_index
    tgt_idx2word = target_embeddings.index_to_key

    # Prepare test data
    test_src_indices = []
    test_tgt_words = []
    for src_word, tgt_word in test_dict:
        if src_word in src_word2idx:
            test_src_indices.append(src_word2idx[src_word])
            test_tgt_words.append(tgt_word)

    num_batches = (len(test_src_indices) + batch_size - 1) // batch_size

    for i in range(num_batches):
        batch_start = i * batch_size
        batch_end = min((i + 1) * batch_size, len(test_src_indices))
        src_indices_batch = test_src_indices[batch_start:batch_end]
        src_vecs = src_vectors[src_indices_batch]
        src_csls_sim = src_avg_sim[src_indices_batch]

        # Compute CSLS similarities
        sims = src_vecs @ tgt_vectors.T
        csls_sims = 2 * sims - src_csls_sim[:, None] - tgt_avg_sim[None, :]

        # For each source word in the batch
        for j in range(csls_sims.shape[0]):
            sims_row = csls_sims[j]
            top_k_indices = np.argpartition(-sims_row, range(k))[:k]
            top_k_words = [tgt_idx2word[idx] for idx in top_k_indices]
            tgt_word = test_tgt_words[batch_start + j]
            if tgt_word in top_k_words:
                correct += 1
            total += 1

    precision = correct / total if total > 0 else 0
    return precision

# Evaluate
p_at_1_csls = precision_at_k_csls(en_mapped, hi_embeddings, test_dict, src_avg_sim, tgt_avg_sim, k=1, batch_size=1024)
p_at_5_csls = precision_at_k_csls(en_mapped, hi_embeddings, test_dict, src_avg_sim, tgt_avg_sim, k=5, batch_size=1024)

print(f'Unsupervised Alignment with CSLS - Precision@1: {p_at_1_csls:.4f}')
print(f'Unsupervised Alignment with CSLS - Precision@5: {p_at_5_csls:.4f}')

Unsupervised Alignment with CSLS - Precision@1: 0.0000
Unsupervised Alignment with CSLS - Precision@5: 0.0000
